# Hierarchial algorithm

Hierarchial algorithm is the technique to organize data in a tree-view shape like the image below:

---

 ![](assets/dendrogram.webp)

---

The diagram above is called a dendrogram. In this tree-view, every node is a cluster for itself. So we can take advantage of not selecting any random "**_K_**" and test which one is better. we can always cut the tree to match our needed clusters count!

There are 2 ways to make this tree. one is called **_Divisive_** and the other is called **_Agglomerative_**.

The only difference is where we start our tree and continue. In **_Agglomerative_**, we go from down to up and in the **_Divisive_**, we start from the very top side to the bottom of the tree.

---

## A quick example of how it works

Imagine we have a dataset like this. This is the distance of 7 cities in Iran.

| | Tehran (TE) | Isfahan (IS) | Shiraz (SH) | Mashhad (MA) | Tabriz (TB) | Yazd (YA) | Ahvaz (AH) |
| :--- | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| **Tehran (TE)** | 0 | 450 | 930 | 890 | 630 | 620 | 800 |
| **Isfahan (IS)** | | 0 | 480 | 1130 | 880 | 320 | 500 |
| **Shiraz (SH)** | | | 0 | 1350 | 1360 | 430 | 540 |
| **Mashhad (MA)**| | | | 0 | 1500 | 900 | 1550 |
| **Tabriz (TB)** | | | | | 0 | 1250 | 1100 |
| **Yazd (YA)** | | | | | | 0 | 800 |
| **Ahvaz (AH)** | | | | | | | 0 |

In this table, what we should do in the Hierarchial algorithm is to find the lowest distance. with a quick check, the lowest distance is between **_Isfahan_** and **_Yazd_**.

Then we will create the table again, but this time, we have just combined **_Isfahan_** and **_Yazd_** together.

---

| | Tehran (TE) | **(IS-YA)** | Shiraz (SH) | Mashhad (MA) | Tabriz (TB) | Ahvaz (AH) |
| :--- | :---: | :---: | :---: | :---: | :---: | :---: |
| **Tehran (TE)** | 0 | 450 | 930 | 890 | 630 | 800 |
| **(IS-YA)** | | 0 | 430 | 900 | 880 | 500 |
| **Shiraz (SH)** | | | 0 | 1350 | 1360 | 540 |
| **Mashhad (MA)**| | | | 0 | 1500 | 1550 |
| **Tabriz (TB)** | | | | | 0 | 1100 |
| **Ahvaz (AH)** | | | | | | 0 |

---

So, now we should check all of the distances again. but the question is how we calculate the distance between **_Tehran_** and **_(IS-YA)_**?

Well, there are several ways to do this, known as **"Linkage Methods."** The method used to create the table above is the simplest one, called **Single Linkage**. It defines the distance from a city (like Tehran) to the new cluster `(IS-YA)` as the **minimum** of the distances to the original cities. For example:
>
> `Distance(Tehran, (IS-YA)) = min(Distance(Tehran, IS), Distance(Tehran, YA))`
>
> `Distance(Tehran, (IS-YA)) = min(450, 620) = 450`

Note that there are 2 more common ways to calculate this distance, which are called **Complete Linkage** and **Average Linkage**. Together, these three are the most fundamental "Linkage Methods."

### 1. Complete Linkage (MAX)

This method is the exact opposite of Single Linkage. It defines the distance from a city to the new cluster as the **maximum** distance to any of its members. It's more conservative and looks at the furthest possible points to determine the cluster distance.

Using our example:

`Distance(Tehran, (IS-YA)) = max(Distance(Tehran, IS), Distance(Tehran, YA))`

`Distance(Tehran, (IS-YA)) = max(450, 620) = 620`

*   **Impact:** This method tends to produce more compact, spherical clusters and is much less sensitive to outliers than Single Linkage.

### 2. Average Linkage (MEAN)

This method provides a balance between the other two. It defines the distance from a city to the new cluster as the **average** of the distances to all of its members.

Using our example:

`Distance(Tehran, (IS-YA)) = (Distance(Tehran, IS) + Distance(Tehran, YA)) / 2`

`Distance(Tehran, (IS-YA)) = (450 + 620) / 2 = 535`

*   **Impact:** This method is less affected by outliers than Single Linkage and often provides a good, balanced result.

---

> "This process of finding the closest pair and merging them is repeated until only one single cluster, containing all the cities, remains. The entire history of these merges creates a tree-like structure that we can then analyze."

---

This algorithm is quite heavy and may take long runtimes as we are doing the same thing over and over again on each node. so this might cost a fortune of resources on a very large dataset! But from the other hand, the answer is always the same. No need to double check.

Enough **theory**! Let's dive into a real world code to see what's going on.

Imagine that an automobile manufacturer has developed prototypes for a new vehicle. Before introducing the new model into its range, the manufacturer wants to determine which existing vehicles on the market are most like the prototypes--that is, how vehicles can be grouped, which group is the most similar with the model, and therefore which models they will be competing against.

Our objective here, is to use clustering methods, to find the most distinctive clusters of vehicles. It will summarize the existing vehicles and help manufacturers to make decision about the supply of new models.

### Let's load the data

In [ ]:
import pandas as pd

filename = 'data/cars.csv'

pdf = pd.read_csv(filename)
print ("Shape of dataset: ", pdf.shape)

pdf.head()

The feature sets include  price in thousands (price), engine size (engine_s), horsepower (horsepow), wheelbase (wheelbas), width (width), length (length), curb weight (curb_wgt), fuel capacity (fuel_cap) and fuel efficiency (mpg).

### Data cleaning

Let's clean the dataset by dropping the rows that have null value:

In [ ]:
print ("Shape of dataset before cleaning: ", pdf.size)
pdf[[ 'sales', 'resale', 'type', 'price', 'engine_s',
       'horsepow', 'wheelbas', 'width', 'length', 'curb_wgt', 'fuel_cap',
       'mpg', 'lnsales']] = pdf[['sales', 'resale', 'type', 'price', 'engine_s',
       'horsepow', 'wheelbas', 'width', 'length', 'curb_wgt', 'fuel_cap',
       'mpg', 'lnsales']].apply(pd.to_numeric, errors='coerce')
pdf = pdf.dropna()
pdf = pdf.reset_index(drop=True)
print ("Shape of dataset after cleaning: ", pdf.size)
pdf.head()

### Feature selection

Let's select our feature set:

In [ ]:
featureset = pdf[['engine_s',  'horsepow', 'wheelbas', 'width', 'length', 'curb_wgt', 'fuel_cap', 'mpg']]

### Normalization

Now we can normalize the feature set. **MinMaxScaler** transforms features by scaling each feature to a given range. It is by default (0, 1). That is, this estimator scales and translates each feature individually such that it is between zero and one.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
x = featureset.values #returns a numpy array
min_max_scaler = MinMaxScaler()
feature_mtx = min_max_scaler.fit_transform(x)
feature_mtx [0:5]

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
dist_matrix = euclidean_distances(feature_mtx,feature_mtx)
print(dist_matrix)

In [ ]:
from scipy.cluster import hierarchy

Z_using_dist_matrix = hierarchy.linkage(dist_matrix, 'complete')

In [ ]:
import pylab

fig = pylab.figure(figsize=(18,50))
def llf(id):
    return '[%s %s %s]' % (pdf['manufact'][id], pdf['model'][id], int(float(pdf['type'][id])) )

dendro = hierarchy.dendrogram(Z_using_dist_matrix,  leaf_label_func=llf, leaf_rotation=0, leaf_font_size =12, orientation = 'right')

In the next step, we can use the 'AgglomerativeClustering' function from scikit-learn library to cluster the dataset.

*   Ward minimizes the sum of squared differences within all clusters. It is a variance-minimizing approach and in this sense is similar to the k-means objective function but tackled with an agglomerative hierarchical approach.
*   Maximum or complete linkage minimizes the maximum distance between observations of pairs of clusters.
*   Average linkage minimizes the average of the distances between all observations of pairs of clusters.


In [ ]:
from sklearn.cluster import AgglomerativeClustering

agglom = AgglomerativeClustering(n_clusters = 6, linkage = 'complete')
agglom.fit(dist_matrix)

agglom.labels_

In [ ]:
pdf['cluster_'] = agglom.labels_
pdf.head()

In [ ]:
pdf.groupby(['cluster_','type'])['cluster_'].count()

In [ ]:
agg_cars = pdf.groupby(['cluster_','type'])[['horsepow','engine_s','mpg','price']].mean()
agg_cars

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(16,10))
for color, label in zip(colors, cluster_labels):
    subset = agg_cars.loc[(label,),]
    for i in subset.index:
        plt.text(subset.loc[i][0]+5, subset.loc[i][2], 'type='+str(int(i)) + ', price='+str(int(subset.loc[i][3]))+'k')
    plt.scatter(subset.horsepow, subset.mpg, s=subset.price*20, c=color, label='cluster'+str(label))
plt.legend()
plt.title('Clusters')
plt.xlabel('horsepow')
plt.ylabel('mpg')

That was all about hierarchical. now let's dive into DBSCAN method!